# Packaging an Overlay

This notebook will demonstrate how to package an Overlay. This notebook depends on the previous three notebooks:

   1. **[Downloading Dependencies](1-Downloading-And-Configuring.ipynb)**
   2. **[Creating a Bitstream](2-Creating-A-Bitstream.ipynb)**
   3. **[Compiling the RISC-V GCC Toolchain](3-Compiling-RISC-V-GCC-Toolchain)**


## Outputs from **[Creating a Bitstream](2-Creating-A-Bitstream.ipynb)**

The first two critical components of a PYNQ overlay are a `.tcl` script file and a bitfile. These files should have been created in **[Creating a Bitstream](2-Creating-A-Bitstream.ipynb)** and with the names `tutorial.tcl` and `tutorial.bit`.

Verify that the following files are in the `<Path-To-RISC-V-On-PYNQ>/riscvonpynq/picorv32/tut/` folder of your PYNQ-RISC-V-Tutorial repository on your host computer. 
       
   1. tutorial.tcl (With the picorv32 IP block and all other associated IP)
   2. tutorial.bit (Compiled using the tutorial.tcl file)
   
Using [SAMBA](http://pynq.readthedocs.io/en/v2.0/getting_started.html#accessing-files-on-the-board), or SCP, copy these files from your host machine to the directory `/home/xilinx/RISC-V-On-PYNQ/tutorial/riscv/` on your PYNQ board.

## Python Overlay Files

In this step we will create the two Python components of an Overlay: The RISC-V Overlay class definition (`tutorial.py`) and the Python Package (`__init__.py`) file.

### Creating a RISC-V Overlay class definition:

In your preferred editor create a new `tutorial.py` file in `<Path-To-RISC-V-On-PYNQ>/riscvonpynq/picorv32/tut/`. You can do this using the [SAMBA](http://pynq.readthedocs.io/en/v2.0/getting_started.html#accessing-files-on-the-board), over SSH, or a [Terminal interface](http://pynq-ucsd.ucsd.edu:9090/terminals/3). 

Using your preferred method, insert the code in the following cell into `tutorial.py`

In [ ]:
from pynq import Overlay, GPIO, Register
import os
import inspect

class TutorialOverlay(Overlay):
    """Overlay driver for the PicoRV32 bram Overlay

    Note
    ----
    This class definition must be co-located with the .tcl and .bit
    file for the overlay for the search path modifications in
    riscvonpynq.Overlay to work. __init__ in riscvonpynq.Overlay uses
    the path of this file to search for the .bit file using the
    inspect package.

    """
    pass

class TutorialProcessor(BramProcessor):
    """Hierarchy driver for the PicoRV32 BRAM Processor

    Note
    ----
    In order to be recognized as a RISC-V Processor hierarchy, three
    conditions must be met: First, there must be a PS-Memory-Mapped
    Block RAM Controller where the name matches the variable
    _bram. Second, the hierarchy name (fullpath) must equal the
    variable _name. Finally, there must be a GPIO port with the name
    _reset_name.

    Subclasses of this module are responsible for setting _name (The
    name of the Hierarchy), _bits (Processor bit-width), _proc
    (Processor Type Name)

    This class must be placed in a known location relative to the
    build files for this processor. The relative path can be modified
    in __get_path.

    """
    _name = 'tutorialProcessor'
    _proc = 'picorv32'
    _bits = 32

    @classmethod
    def checkhierarchy(cls, description):
        return super().checkhierarchy(description)

    def __get_path(self):
        """Get the directory path of this file, or the directory path of the
        class that inherits from this class.

        """
        # Get file path of the current class (i.e. /opt/python3.6/<...>/stream.py)
        file_path = os.path.abspath(inspect.getfile(inspect.getmodule(self)))
        # Get directory path of the current class (i.e. /opt/python3.6/<...>/stream/)
        return os.path.dirname(file_path)

    def __init__(self, description, *args):
        """Return a new Processor object. 

        Parameters
        ----------
        description : dict
            Dictionary describing this processor.

        """
        build_path = os.path.join(self.__get_path(), "build")
        reset_value = 0
        super().__init__(build_path, reset_value, description, *args)

### Creating an \__init.py\__ file

The `__init__.py` file is a simple file that structures packages within the python namespace. 

In your preferred editor create a new `__init__.py` file in the `/home/xilinx/PYNQ-RISC-V/tutorial/riscv/notebooks` folder of the PYNQ-RISC-V repository on your PYNQ Board. Insert the code from the following cell:

In [ ]:
from . import tutorial
from . import build

Save and close the file. 

For more information on `__init__.py` files see the [Python Documentation](https://docs.python.org/3/tutorial/modules.html#packages)

## Build Files 

Finally, we need to create a `build` folder and populate its contents.

To do this, run the following cell:

In [ ]:
!cp /home/xilinx/PYNQ-RISC-V/riscvonpynq/picorv32/bram/build/ /home/xilinx/PYNQ-RISC-V/riscvonpynq/picorv32/tut/build

This will create build files for your processor. 

### Including the Tutorial processor in RISC-V-On-PYNQ

To include your processor in the pip installation modify `__init__.py` in `/home/xilinx/PYNQ-RISC-V/riscvonpynq/picorv32/bram/build` with the text shown below:

In [ ]:
from . import axi
from . import bram
from . import tut

## Testing the Tutorial Overlay 

Now that the files have been made we need to verify that they are correct. Everything that we have created gives us a working overlay.

Now, for the first time, try loading your overlay by running the following cell:

<div class="alert alert-block alert-info">
**Note:** The first two lines change the search path for this tutorial for this namespace (i.e. this change is not permanent)
</div>

In [1]:
import sys
sys.path.insert(0, '/home/xilinx/RISC-V-On-PYNQ/riscvonpynq/picorv32/')

from tut.tutorial import TutorialOverlay

overlay = TutorialOverlay("tutorial.bit")

Help on package tut:

NAME
    tut

PACKAGE CONTENTS
    build (package)
    tutorial

FILE
    /home/xilinx/RISC-V-On-PYNQ/riscvonpynq/picorv32/tut/__init__.py


Help on TutorialProcessor in module tut.tutorial object:

class TutorialProcessor(riscvonpynq.Processor.BramProcessor)
 |  Hierarchy driver for the PicoRV32 BRAM Processor
 |  
 |  Note
 |  ----
 |  In order to be recognized as a RISC-V Processor hierarchy, three
 |  conditions must be met: First, there must be a PS-Memory-Mapped
 |  Block RAM Controller where the name matches the variable
 |  _bram. Second, the hierarchy name (fullpath) must equal the
 |  variable _name. Finally, there must be a GPIO port with the name
 |  _reset_name.
 |  
 |  Subclasses of this module are responsible for setting _name (The
 |  name of the Hierarchy), _bits (Processor bit-width), _proc
 |  (Processor Type Name)
 |  
 |  This class must be placed in a known location relative to the
 |  build files for this processor. The relative path can be

If the cell above fails, check that you have completed the steps above correctly.

To test code on your processor, run the following Jupyter magic. This cell defines a main method. It expects two arguments. The first argument is the path of the file on the ARM PS. The second argument is an array of at least three elements. 

The program returns the third element of the second argument.

Run the following cell to compile it. If the cell fails, check that `<Path-To-RISC-V-On-PYNQ>/riscvonpynq/picorv32/tut/build` exists.

In [2]:
%%riscvc test overlay.tutorialProcessor

int main(int argc, char ** argv){
    unsigned int * arr = (unsigned int *)argv[1];
    return arr[2];
}

To run the program above, execute the following cell. The first argument will automatically be created by the `run` method of the tutorialProcessor

In [6]:
import numpy as np
arg1 = np.array([4,2,3], np.uint32)

retval = overlay.tutorialProcessor.run(test, arg1)

if(retval != arg1[2]):
    print("Test failed!")
else:
    print("Test passed!")

Test passed!


If the test passed, congratulations!

# Installing the Tutorial Overlay

By modifiying the `__init__.py` files in `<Path-To-RISC-V-On-PYNQ>/riscvonpynq/picorv32/` and `<Path-To-RISC-V-On-PYNQ>/riscvonpynq/picorv32/tut` we have added the overlay created in this tutorial to the `riscvonpynq` package in the local repository. To install it formally, run the following cell: 

In [ ]:
!pip3.6 install --upgrade /home/xilinx/RISC-V-On-PYNQ/

If the cell above completes successfully, you can now run the previous cells without modifying the system path:

In [ ]:
from riscvonpynq.picorv32.tut.tutorial import TutorialOverlay

overlay = TutorialOverlay("tutorial.bit")

In [ ]:
%%riscvc test overlay.tutorialProcessor

int main(int argc, char ** argv){
    unsigned int * arr = (unsigned int *)argv[1];
    return arr[2];
}

In [ ]:
import numpy as np
arg1 = np.array([4,2,3], np.uint32)

retval = overlay.tutorialProcessor.run(test, arg1)

if(retval != arg1[2]):
    print("Test failed!")
else:
    print("Test passed!")